In [8]:
import pandas as pd
import numpy as np
from darts import TimeSeries
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from mlflow import MlflowClient 
from mlflow.models import infer_signature
import mlflow

In [11]:
#df = pd.read_csv(r"C:\Users\jakob\Downloads\archive (1)\AirPassengers.csv")

ab_df = pd.read_csv('../../data/interim/absences_per_day.csv', sep=',')


In [12]:
ab_df = ab_df[['index', 'count']]
# drop all rows with index older than 2023-03-01
ab_df = ab_df.drop(ab_df[ab_df['index'] > '2023-03-01'].index)
# normalize data   
#ab_df['count'] = ab_df['count'] / ab_df['count'].max()


In [10]:
ab_df

,index,count
0,2020-01-02,0.004082
1,2020-01-03,0.004082
2,2020-01-04,0.004082
3,2020-01-05,0.002041
4,2020-01-06,0.002041
...,...,...
1150,2023-02-25,0.440816
1151,2023-02-26,0.138776
1152,2023-02-27,0.232653
1153,2023-02-28,0.234694


In [13]:
#series = TimeSeries.from_dataframe(df, 'Month', '#Passengers')
#series = TimeSeries.from_dataframe(df, 'Month', '#Passengers')
#series
#creaet new dataframe with only date and count
ab_df['index'] = pd.to_datetime(ab_df['index'])

series = TimeSeries.from_dataframe(ab_df, 'index', 'count')


In [18]:
#from darts.models import ExponentialSmoothing

#train_ts, test_ts = series.split_before(pd.Timestamp('20220701'))

#model = ExponentialSmoothing()
#model.fit(train_ts)
#prediction = model.predict(len(test_ts))

In [ ]:
#import matplotlib.pyplot as plt

#series.plot(label='actual')
#prediction.plot(label='forecast', lw=3)
#plt.legend()

In [22]:
from darts.models import ExponentialSmoothing
from darts.models import Prophet


#models = [ExponentialSmoothing(), Prophet()]

models = [ExponentialSmoothing()]

backtests = [model.historical_forecasts(series,
                            start=.7,
                            forecast_horizon=3)
             for model in models]



In [23]:
single_train_ts, single_test_ts = series.split_before(pd.Timestamp('2023-01-01'))


In [25]:
from darts.dataprocessing.transformers import Scaler, StaticCovariatesTransformer

pipeline = Scaler()
#pipeline = Pipeline([Scaler(), StaticCovariatesTransformer()]) # MinMaxScaler
train_dataset_ts_prepared = pipeline.fit_transform(single_train_ts)
test_dataset_ts_prepared = pipeline.transform(single_test_ts)

In [26]:
forecast_horizons = len(single_test_ts)

In [29]:
model_ExponentialSmoothing = ExponentialSmoothing(lags=2, 
                            output_chunk_length=356, 
                            random_state=0,
                            multi_models=False, 
                            #lags_past_covariates=past_lags,
                            #lags_future_covariates=future_lags,
                            add_encoders={"cyclic": {"future": ["month"]}, 
                                         'datetime_attribute': {'future': ['dayofweek']},
                                         'position': {'past': ['relative'], 'future': ['relative']},}
                                         #'transformer': Scaler()}
                            )

model_ExponentialSmoothing.fit(series=train_dataset_ts_prepared)
prediction_ExponentialSmoothing = model_ExponentialSmoothing.predict(n=forecast_horizons, series=train_dataset_ts_prepared)
prediction_ExponentialSmoothing = pipeline.inverse_transform(prediction_ExponentialSmoothing)

TypeError: fit() got an unexpected keyword argument 'output_chunk_length'

In [ ]:
prediction_prophet = Prophet.predict(n=forecast_horizons, series = train_ts)
#prediction_prophet = pipeline.inverse_transform(prediction_lightGBM)

In [ ]:
from darts.metrics import mape

series.plot(label='data')
for i, m in enumerate(models):
    err = mape(backtests[i], series)
    backtests[i].plot(lw=3, label='{}, MAPE={:.2f}%'.format(m, err))

plt.title('Backtesting results')
plt.legend()

In [ ]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [ ]:
mlflow.set_tracking_uri('./mlruns')
mlflow.set_experiment('singlets')
experiment = mlflow.get_experiment_by_name('singlets')
client = MlflowClient()

run = client.create_run(experiment.experiment_id)

with mlflow.start_run(run_id=run.info.run_id) as run:

    (rmse, mae, r2) = eval_metrics(y_test, y_pred)
    
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    mlflow.prophet.log_model(prophet, 'model')